# 🧹 Data Cleaning and Exploratory Analysis

In this notebook, we'll dive deep into the candidate data, cleaning it, analyzing patterns, and preparing it for further use.

## 🚀 Steps Overview

1. **Database Connection** 🔗
2. **Exploratory Data Analysis (EDA)** 📊
3. **Data Cleaning and Transformation** 🧼



---

## 1. 🔗 Database Connection

We start by connecting to the PostgreSQL database to retrieve the raw data for analysis.



In [3]:
import sys
import os
from dotenv import load_dotenv

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener el directorio de trabajo desde las variables de entorno
work_dir = os.getenv('WORK_DIR')

# Añadir el directorio de trabajo al sys.path
sys.path.append(work_dir)


In [4]:
from src.db_connection import build_engine
from src.transform_data import DataTransformer


In [7]:
from sqlalchemy import inspect
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError

In [5]:
# Reuse the connection to the PostgreSQL database
engine = build_engine()

Successfully connected to the database postgres!


---
## 2. 📊 Exploratory Data Analysis (EDA)
Here, we explore the dataset to understand its structure, identify any inconsistencies, and visualize key patterns.

### 2.1 Basic Data Overview

In [6]:
# Load data from the Candidates_raw table
df = pd.read_sql('SELECT * FROM Candidates_raw', con=engine)
df.head()


NameError: name 'pd' is not defined